In [ ]:
! pip install -r requirements.txt

In [ ]:
from labeler_ui import Authentication, Dashboard
from labeler_client import Service
import pandas as pd
import os

# Connecting to the service
We provide a shared project to simuliate the real-world collaboration among data science practitioners. This project is pre-loaded with a small set of [tweets about US airlines](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment). Required labeling schema and meta-data are also pre-loaded (shown as screen shots in later cells). Please do not change these setups.

To start, connect to our service using one of our demo users's token. Note token sharing is for the demostration purposes only, and might cause potential overwrite of other's annotation. In real projects, each user own their unique token.

**Run the cells below to connect to the demo project**

In [ ]:
# getting demo token.
demo_user_id=5 # choose between 1-5
tokens = pd.read_csv('tokens.csv',index_col='id')
demo_token = tokens.loc[demo_user_id,'token'] 

In [ ]:
# connecting to service
demo = Service(project='development',token=demo_token,
               host='http://labeler-demo-service-alb-1400746965.us-west-1.elb.amazonaws.com')

# View label schema
Here the schema is pre-set to collect one label named *sentiment* which corresponds to a classification task and a label named *sp* which correponds to an extraction task (selecting spans out of a document).

**Run the cell below to check the schema**

In [ ]:
# view the pre-set demo schema
demo.get_schemas().value(active=True)

# Try annotating

Here we show the basic annoation widget.

**Run the cell below to bring up the widget to annotate the first 10 examples**
You can switch between the *single* and *table* view on the top-right corner. Switch back to the table view, check the top checkbox on the second column and click **submit** button on the top-left corner to send your annotations to the backend. 

You can also click the *Annotating* button with a dropdown on the top-right corner to switch between annotation and reconciling mode. In the reconciling mode, you could look over distributions of existing annotations from all annotators and resolve conflicts.

(Note some data might already have annotations if someone else sharing the same token already annotated the data point.)

In [ ]:
# search results => subset s1
s1 = demo.search(keyword='', limit=10, start=0)
# bring up a widget 
s1.show({'view':'table'})

# Show contexual data
There are cases when contextual information could help or speed up the annotatin process. Here we demonstrate with a pre-loaded "hashtag" metadata, which was generated using the code shown in this screenshot:
![generating the hashtag meta-data](Figures/hashtag.png)

In real projects, the metadata could be generated using user-defined functions, like in the screenshot.

**Run the cell below to see how hashtags are shown as auxiliary information**

In [ ]:
s2= demo.search(keyword='delay',
                  limit=50,
                  start=0, 
                  meta_names=['hashtag'])
s2.show()
#hover over in the table view to see the hashtage for each data ponint.

# Exploratory annotation: heuristic-based search

In [ ]:
s3= demo.search(keyword='fail',
                  limit=50,
                  start=0)
s3.show({'view':'table'})

# Exploratory annotation: similarity-based suggestions 
In addition to relying on user's heuristic, meganno also provide automated suggestions, based on all kinds of metadata. Here we give an example using sentence bert embeddings. The embeddings have been generated ahead of time using the script shown in the screenshot below:

![generating bert embedding](./Figures/bert.png)
Based on the embedding, suggest the most similar datapoints from the database.

In real projects, users can bring
*Try Below*

In [ ]:
s4 = demo.search(keyword='delay', limit=3, start=0)
# try replacing the keyword here to test the similarity search 
s5 = s4.suggest_similar('bert-embedding', limit=4)# needs to provide a valid meta_name
s5.show()

# Analysis Dashboard to track project progress.
At any stage of the project, you can run the cell below to track the progress of all annnotators, look at class distibutions, etc

In [ ]:
#Analysis
from labeler_ui import Dashboard
dash_wg = Dashboard(demo)
dash_wg.show()


In [ ]:
# exporting all annotations to csv if need to bring data out.
demo.export()